In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

# Workbook

| Date       | Version   | Description                                      | Public score  | Commit Version
|--------------------------------------------------------------------------------------------------------------------------------------------|
| 2019-06-01 | v1.0        | resnet50 transfer learning, loss='categorical_crossentropy' batch-size: 64   | -  | v6 |
| 2019-06-07 | v1.1        | resnet50 transfer learning,  kappa score failed.   |   -| v7 |
| 2019-06-10 | v1.2        | resnet50 transfer learning,  batch_size: 32 (as in book p. 321) |   -| v7 |
| 2019-06-10 | v1.3        | resnet50 transfer learning,  batch_size: 32, dropout rate 0.2 |   -| v8 |
| 2019-06-11 | v1.4        | resnet50 transfer learning,  batch_size: 32, dropout rate 0.5, EarlyStop 2 |   -| v9 |
| 2019-06-15 | v1.5        | resnet50 transfer learning,  batch_size: 32, dropout rate 0.5, EarlyStop 10  | -| v10 |
| 2019-06-16 | v1.6        | resnet50 transfer learning,  batch_size: 512, dropout rate 0.5, EarlyStop 10, adam | -| https://www.kaggle.com/bravenoob/fork-of-mas-keras?scriptVersionId=15757244 |
| 2019-06-16 | v1.7        | check kappa   |  -| v11 |
| 2019-06-20 | v1.8        | submit test (no 0's were concluded)   |  0.25981 | v12 |
| 2019-06-20 | v1.9        | submit test from best talos experiment params (64, 0.5, 50, 'softmax', 2.08,'categorical_crossentropy', 0.25,Adam') | 0.17356 | 16 |
| 2019-06-27 | v1.10        | submit test with batch size 512 (0.2, 20, 'softmax', 'None', 0.1, Adam, 512, 'categorical_crossentropy', 0.1, 0] | 0.23107 | v17 |
 | 2019-06-27 | v1.11       | same, but 50 epochs & early stopping (mean) |   0.22490 | v18 |
 | 2019-07-07 | v1.12       | same, classifier matrix prediction | 0.23852 | https://www.kaggle.com/bravenoob/mas-keras-dataframe?scriptVersionId=17063755 |
 | 2019-07-11 | v1.13       | NASNet large, default params, new image folder structure |  0.29894 | v20 |
 | 2019-07-11 | v1.14       | NASNet large, default params,, class_weights, early stopping  |  0.31138 | v21 |
 | 2019-07-13 | v1.15       | same, classifier matrix prediction  |  0.28453 | https://www.kaggle.com/bravenoob/mas-keras-dataframe?scriptVersionId=17157275 |
 | 2019-07-14 | v1.16       | NASNet large, talos params, no class_weights, epoch 20,batch: 16, dropout 0.1, learningrate:0.00109  | 0.25280 | v23 |
 | 2019-07-14 | v1.17       | new layers |  0.19965 | https://www.kaggle.com/bravenoob/mas-keras-kappa?scriptVersionId=17469784 |
 | 2019-07-19 | v1.19       | new layers, with class weight |  0.26575 | v35 |
 | 2019-07-20 | v1.20       | new layers, with class weight, add image augmentation |  0.29932 | v36 
 | 2019-07-21 | v1.21       | new layers, with class weight, add image augmentation (new image split), very promising lcurve, more time needed |  0.30489 | v42 
 | 2019-07-25 | v1.22       | same, classifier matrix prediction, worse than just mean, but why? | 0.28440 | https://www.kaggle.com/bravenoob/mas-keras-dataframe?scriptVersionId=17774299 
 | 2019-07-27 | v1.23       | v1.21 on vast.ai 100 epochs (data: vast.ai submission v2) | 0.28782 | https://www.kaggle.com/bravenoob/mas-keras-dataframe?scriptVersionId=17774299 
 | 2019-07-28 | v1.24       | max pooling sieht schlecht aus | - | v43 
 | 2019-07-29 | v1.25       | vast.ai 100 epochs, mehr augementation (data: vast.ai submission v3) | 0.26451 | code verzeichnis 
 | 2019-08-02 | v1.26       | params from v.1.21 with resized images | 0.30519 | v47 
 | 2019-08-04 | v1.27       | matrix classifier | 0.27413 | https://www.kaggle.com/bravenoob/mas-keras-dataframe?scriptVersionId=18264736
 | 2019-08-02 | v1.28       | best talos score lr,batch_size,dropout,optimizer 0.1,32,0.1, Adamax | fail | v50
 | 2019-08-12 | v1.29       | best talos score lr,batch_size,dropout,optimizer 0.1,64,0.25, Adam | - | v52

In [ ]:
# Load the extension and start TensorBoard
%load_ext tensorboard.notebook
%tensorboard --logdir logs

In [ ]:
import time 

#let's also import the abstract base class for our callback
from tensorflow.keras.callbacks import Callback

#defining the callback
class TimerCallback(Callback):
    
    def __init__(self, maxExecutionTime):
        
# Arguments:
#     maxExecutionTime (number): Time in minutes. The model will keep training 
#                                until shortly before this limit
#                                (If you need safety, provide a time with a certain tolerance)
        
        self.maxExecutionTime = maxExecutionTime * 60
    
    
    #Keras will call this when training begins
    def on_train_begin(self, logs):
        self.startTime = time.time()
        self.longestTime = 0            #time taken by the longest epoch or batch
        self.lastTime = self.startTime  #time when the last trained epoch or batch was finished

    #this is our custom handler that will be used in place of the keras methods:
        #`on_batch_end(batch,logs)` or `on_epoch_end(epoch,logs)`
    def on_epoch_end(self, epoch, logs):
        
        currentTime      = time.time()                           
        self.elapsedTime = currentTime - self.startTime    #total time taken until now
        thisTime         = currentTime - self.lastTime     #time taken for the current epoch
                                                               #or batch to finish
        
        self.lastTime = currentTime
        
        #verifications will be made based on the longest epoch or batch
        if thisTime > self.longestTime:
            self.longestTime = thisTime
        
        
        #if the (assumed) time taken by the next epoch or batch is greater than the
            #remaining time, stop training
        remainingTime = self.maxExecutionTime - self.elapsedTime
        if remainingTime < self.longestTime:
            
            self.model.stop_training = True  #this tells Keras to not continue training
            print("\n\nTimerCallback: Finishing model training before it takes too much time. (Elapsed time: " + str(self.elapsedTime/60.) + " minutes )\n\n")

In [ ]:
train_df = pd.read_csv("../input/petfinder-adoption-prediction/train/train.csv")

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
from tensorflow.keras.applications import NASNetLarge
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam, Nadam, Adamax

num_classes = 5
model = NASNetLarge(weights='imagenet', include_top=False, pooling='avg')

my_new_model = Sequential()
my_new_model.add(model)
my_new_model.add(Dense(512, activation="relu"))
my_new_model.add(Dropout(rate=0.25))
my_new_model.add(Dense(256, activation="relu"))
my_new_model.add(Dropout(rate=0.25))                 
my_new_model.add(Dense(num_classes, activation='softmax'))

# Say not to train first layer (ResNet) model. It is already trained
my_new_model.layers[0].trainable = False

# optimizer
# descent optimizer (adam lr defaut = 0.001)
my_new_model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

print(my_new_model.summary())

In [ ]:
layers = [(layer, layer.name, layer.trainable) for layer in my_new_model.layers]
pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])   

In [ ]:
from tensorflow.keras.applications.nasnet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.callbacks import Callback

image_size = 331
batch_size = 64
nb_epochs = 50

# steps_per_epoch: number of yields (batches) before a epoch is over
# ceil(num_samples / batch_size)
# epochs: Number of epochs to train the model. An epoch is an iteration over the entire data provided
# class_weight: Optional dictionary mapping class indices (integers) to a weight (float) value, used for weighting the loss function (during training only). 
#  This can be useful to tell the model to "pay more attention" to samples from an under-represented class.

shift = 0.2

#brightness_range=[0.8,1.2],
#zoom_range=[0.8,1.2],
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input, zoom_range=[0.9,1.1],width_shift_range=0.2, 
                                       height_shift_range=0.2, horizontal_flip=True)

#liefert die trainingsdaten als iterator
# image aug funktioniert so, dass für den aktuellen batch die bilder geändert werden. Nicht dass es mehr Bilder gibt.
# Somit wird für jede Epoche mit anderen Bildenr trainiert.
train_generator = train_datagen.flow_from_directory(
    '../input/petfinder-images/petfinder_images/images/train',
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='categorical')

# keine image augmenation für validierung
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

#liefert die Validationdaten als iterator
validation_generator = test_datagen.flow_from_directory(
    '../input/petfinder-images/petfinder_images/images/validation',
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='categorical')

STEP_SIZE_TRAIN=math.ceil(train_generator.n//train_generator.batch_size)
STEP_SIZE_VALID=math.ceil(validation_generator.n//validation_generator.batch_size)

print(STEP_SIZE_TRAIN)
print(STEP_SIZE_VALID)


# Configure the TensorBoard callback and fit the model
tensorboard_callback = TensorBoard("logs")

# Early stopping against overfit
earlystopping_callback = EarlyStopping(patience=batch_size/10, monitor='val_acc', mode='auto', restore_best_weights=True)
        
# save best model
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='auto', save_best_only=True)

class_weights = {0: 8.80595533,
                1: 1.01597481,
                2: 0.72513282,
                3: 0.74640867,
                4: 0.84505298}


#import multiprocessing
#multiprocessing.cpu_count()

timerCallback = TimerCallback(500)

history = my_new_model.fit_generator(
    train_generator,
    steps_per_epoch = STEP_SIZE_TRAIN,
    validation_data = validation_generator, 
    validation_steps = STEP_SIZE_VALID,
    epochs = nb_epochs,
    class_weight = class_weights,
    callbacks=[mc,timerCallback,earlystopping_callback],
    workers = 2,
    use_multiprocessing = False,
    max_queue_size = 40)

In [ ]:
import matplotlib.pyplot as plt
def plot_training(history):
    # Plot training & validation accuracy values
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

    # Plot training & validation loss values
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

    plt.savefig('acc_vs_epochs.png')
    
plot_training(history)

In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.nasnet import preprocess_input, decode_predictions

saved_model = load_model('../working/best_model.h5')

In [ ]:
import math

def calculate_score_from_predictions(df, PetID):
    mean = df[df['Filename'].str.contains(PetID)]['Predictions'].mean()
    if math.isnan(mean):
        return 4 # if no photo is available, default is Speed 4
    else: 
        return int(round(mean, 0))

In [ ]:
test = pd.read_csv('../input/petfinder-adoption-prediction/test/test.csv')

In [ ]:
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

#liefert die testdaten als iterator
test_generator = test_datagen.flow_from_directory(
    '../input/petfinder-images-331/petfinder_images_331/images_crop/real_test', # expects a single folder within
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False)

test_generator.reset()

test_preds = saved_model.predict_generator(test_generator)
test_results=pd.DataFrame({"Filename":test_generator.filenames,
                      "Predictions":test_preds.argmax(axis=-1)})

In [ ]:
print(test_results.shape)
test_results.Predictions.value_counts()

In [ ]:
submit=pd.DataFrame()
submit['PetID']=test['PetID']
submit['AdoptionSpeed']=test['PetID'].apply(lambda x: calculate_score_from_predictions(test_results, x))
submit.to_csv('submission.csv',index=False)